In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import os 
#os.environ["CUDA_VISIBLE_DEVICES"] = '1'
import sys
import copy 
import datetime as dt
import json
import re
import os
import time as t
import gzip

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

import torchvision
import torchvision.transforms as transforms
from   torchvision.transforms import Compose, ToTensor, Resize

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn
from   sklearn.metrics import confusion_matrix,r2_score
from   sklearn.svm import SVC
from   sklearn.pipeline import make_pipeline
from   sklearn.preprocessing import StandardScaler
from   sklearn.model_selection import train_test_split
from   sklearn import tree
from   sklearn.ensemble import RandomForestClassifier
from   sklearn.utils import shuffle
from   sklearn.decomposition import PCA
from sklearn.isotonic import IsotonicRegression
from sklearn.datasets import load_boston, load_diabetes, fetch_california_housing

import scipy.stats as stats
from   scipy.stats import moment, kurtosis, skew, norm, kstest, wasserstein_distance
from scipy.interpolate import interp1d

import seaborn as sbn

%matplotlib inline

# Network, Metrics, ...

In [ ]:

def cov_matrix(c,n):
    return (1-c) * np.diag(np.ones(n)) + c * np.ones((n,n))

def corr_init_matrix(in_features,out_features,c):
    
    cdf_func = lambda x: norm.cdf(x,loc=0,scale=np.sqrt(2)/2)
    n        = max(in_features,out_features)
    
    W  = np.random.multivariate_normal(np.zeros(n),cov_matrix(c,n),n)
    W += np.random.multivariate_normal(np.zeros(n),cov_matrix(c,n),n).T
    W  = 0.5*W
    W  = cdf_func(W)
    
    W  = (2*W-1)*np.sqrt(1.0/in_features)
    W = W[:out_features,:in_features]
    
    return torch.nn.Parameter(torch.FloatTensor(W))

In [ ]:
# network for MC dropout and deep ensembles

In [ ]:
# Fully connected neural network with three hidden layers (with dropout)
class Net(nn.Module):
    def __init__(self, net_params, train_params):
        super(Net, self).__init__()
        
        self.n_input      = net_params['n_input']
        self.layer_width  = net_params['layer_width']
        self.num_layers   = net_params['num_layers']
        self.n_output     = net_params['n_output']
        self.nonlinearity = net_params['nonlinearity']
            
        self.drop_bool    = train_params['drop_bool']
        self.drop_bool_ll = train_params['drop_bool_ll']
        self.drop_p       = train_params['drop_p']
        
        self.layers       = nn.ModuleList()
        
        if self.num_layers == 0:
            self.layers.append(nn.Linear(self.n_input,self.n_output))
        else:
            self.layers.append(nn.Linear(self.n_input,self.layer_width))
            for _ in range(self.num_layers-1):
                self.layers.append(nn.Linear(self.layer_width,self.layer_width))
            self.layers.append(nn.Linear(self.layer_width,self.n_output))
    
        for layer in self.layers:
            layer.weight = corr_init_matrix(layer.in_features,layer.out_features,net_params['init_corrcoef'])

    def forward(self, x, drop_bool=None):
        
        # drop_bool controls whether last layer dropout is used (True/False) or if values from the constructor shall be used (None)
        if drop_bool is None:
            drop_bool    = self.drop_bool
            drop_bool_ll = self.drop_bool_ll
        elif drop_bool is False:
            drop_bool_ll = False
        elif drop_bool is True:
            drop_bool_ll = True
        
        if self.num_layers == 0:
            x = F.dropout(x, p=self.drop_p, training=drop_bool_ll)
            x = self.layers[-1](x)
        else:
            for layer in self.layers[:-2]:
                x = layer(x)
                x = self.nonlinearity(x)
                x = F.dropout(x, p=self.drop_p, training=drop_bool)

            x = self.layers[-2](x)
            x = self.nonlinearity(x)
            x = F.dropout(x, p=self.drop_p, training=drop_bool_ll)

            x = self.layers[-1](x)
        
        return x

In [ ]:
# network for parametric uncertainty (PU)
# x[:, 0]: Network output, x[:, 1] uncertainty estimate

In [ ]:
class Net_PU(Net):
    def __init__(self, net_params, train_params):
        super(Net_PU, self).__init__(net_params=net_params, train_params=train_params)
        self.softplus    = nn.Softplus()
    
    def forward(self, x):
        for layer in self.layers[:-1]:
            x = layer(x)
            x = self.nonlinearity(x)
            x = F.dropout(x, p=self.drop_p, training=self.drop_bool)
        x = self.layers[-1](x)
        x = torch.stack([x[:,0],self.softplus(x[:,1])],dim=1)
        return x     

In [ ]:
def train_network(net,data,train_params,method):

    if method in ['de','pu_de','sml_de']:  # de = deep ensembles; net is a list, train all networks in that list
        for i in range(len(net)):
            train_network(net[i],data=data,train_params=train_params,method='mc')
            
    else:
        
        start_time = t.time()
        
        X_train,y_train = data
        batch_size = train_params['batch_size']
        batch_no = len(X_train) // batch_size

        optimizer = torch.optim.Adam(net.parameters(), lr=train_params['learning_rate'])
        #optimizer = torch.optim.SGD(net.parameters(), lr=0.2)
        loss_func = train_params['loss_func'] 

        running_loss = 0.0
        for epoch in range(train_params['num_epochs']):
            
            X_train, y_train = shuffle(X_train, y_train)
            
            
            start_time_2 = t.time()
            for i in range(batch_no):
                
                start  = i * batch_size
                end    = start + batch_size
                inputs = torch.FloatTensor(X_train[start:end]).to(train_params['device'])
                labels = torch.FloatTensor(y_train[start:end].flatten()).to(train_params['device'])
                labels = torch.unsqueeze(labels,dim=1)

                # zero the parameter gradients
                optimizer.zero_grad()
                
                # forward + backward + optimize
                if loss_func == sml_loss:
                    loss = sml_loss(net=net,data=[inputs,labels],loss_params=train_params['sml_loss_params']) 
                elif loss_func == train_second_moments_loss:
                    loss = train_second_moments_loss(net=net,data=[inputs,labels],loss_params=train_params['sml_loss_params']) 
                else:
                    outputs = net(inputs)
                    loss    = loss_func(outputs,labels)
                
                loss.backward()
                optimizer.step()

                # print statistics
                running_loss += loss.item()
                
            end_time_2 = t.time()
                
            if epoch % 100 == 0:
                end_time = t.time()
                print('Epoch {}'.format(epoch), "loss: ",running_loss, "took: %.5fs (exp. total time: %.5fs)" % (end_time-start_time, (end_time-start_time)*train_params['num_epochs']/100) )
                start_time = t.time()
            running_loss = 0.0

In [ ]:
def calc_datapoint_statistics(net,data,method, iso_reg=None):
    
    X,y = data
    pred_y_samples = []
    eps = 1e-10

    df = pd.DataFrame(y.flatten()).rename(columns={0:'gt'})
    df['x'] = X.tolist()
    
    with torch.no_grad(): 
        
        # Compute mean and std from network outputs   
        if 'mc' in method: # Get predictions with deactivated dropout and multiple predictions per input point with activated dropout
            pred_y_no_mc = list((net(torch.FloatTensor(X),drop_bool=False).cpu().numpy()).flatten()) 
            for _ in range(200):
                pred_y_samples.append(list((net(torch.FloatTensor(X)).cpu().numpy()).flatten()))
            df['pred_mean'] = pd.DataFrame(pred_y_samples).mean()
            df['pred_std']  = pd.DataFrame(pred_y_samples).std()
            
        
        elif method == 'de':
            for i in range(len(net)):
                pred_y_samples.append(list((net[i](torch.FloatTensor(X)).cpu().numpy()).flatten()))
            #df['pred_y_samples'] = list(np.asarray(pred_y_samples).reshape((-1, len(net))))
            df['pred_mean'] = pd.DataFrame(pred_y_samples).mean()
            df['pred_std']  = pd.DataFrame(pred_y_samples).std()
            
        
        elif method == 'pu':
            df[['pred_mean','pred_std']] = pd.DataFrame(net(torch.FloatTensor(X)).cpu().numpy())

            
        elif method == 'pu_de':
            mus = []
            sigmas = []
            for net_ in net:
                net_mu_sigma = net_(torch.FloatTensor(X)).cpu().data.numpy()    
                mus.append(net_mu_sigma[:,0])
                sigmas.append(net_mu_sigma[:,1])

            mus    = np.array(mus)
            sigmas = np.array(sigmas)
            df['pred_mean'] = mus.mean(axis=0)
            df['pred_std']  = np.sqrt( (sigmas**2 + mus**2).mean(axis=0) - df['pred_mean']**2 )

            
        elif method == 'sml_de':
            mus = []
            sigmas = []
            for net_ in net:
                net_pred_no_mc = list((net_(torch.FloatTensor(X),drop_bool=False).cpu().numpy()).flatten())
                pred_y_samples = []

                for _ in range(200):
                    pred_y_samples.append(list((net_(torch.FloatTensor(X)).cpu().numpy()).flatten()))

                net_pred_mean = pd.DataFrame(pred_y_samples).mean()
                net_pred_std  = pd.DataFrame(pred_y_samples).std()
                net_spread    = net_pred_mean - net_pred_no_mc
                net_total_std = net_pred_std + np.abs(net_spread)
                mus.append(net_pred_no_mc)
                sigmas.append(net_total_std)

            mus    = np.array(mus)
            sigmas = np.array(sigmas)
            df['pred_mean'] = mus.mean(axis=0)
            df['pred_std']  = np.sqrt( (sigmas**2 + mus**2).mean(axis=0) - df['pred_mean']**2 )




        if 'mc' in method:
            df['pred_no_mc'] = pred_y_no_mc
            df['spread']     = df['pred_mean'] - df['pred_no_mc']


        # Further metrics: nll (of gt in model under gaussian assumption), residual (i.e. mean - gt), error quantile (quantile of gt in normalized prediction distribution)        
        if method == 'mc_mod_sml':
            df['total_std'] = df['pred_std']+np.abs(df['spread'])
            df['nll']                  = df.apply(lambda x: nll(x['pred_no_mc'],x['total_std'],x['gt']),axis=1)
            df['pred_residual']        = df['pred_no_mc']-df['gt']
        else:
            df['total_std'] = df['pred_std']
            df['nll']                  = df.apply(lambda x: nll(x['pred_mean'],x['pred_std'],x['gt']),axis=1)
            df['pred_residual']        = df['pred_mean']-df['gt']

        df['pred_residual_normed'] = df['pred_residual']/(df['total_std']+eps)   
        df['error_quantile']       = df['pred_residual_normed'].apply(lambda x: np.round(norm.cdf(x),2))



    if 'mc' in method:
        df['net_gradient_norm'] = pd.DataFrame(X).apply(lambda x: net_gradient_norm(x,net),axis=1)
    else:
        df['net_gradient_norm'] = 1e10



    _, iso_reg_ = calc_ece_and_iso_reg(df['error_quantile'])
    if iso_reg is not None:
        if isinstance(iso_reg, list):
            if len(iso_reg) == 0:
                iso_reg.append(iso_reg_)
                df['error_quantile_calibrated'] = iso_reg_.predict(df['error_quantile'])
            else:
                df['error_quantile_calibrated'] = iso_reg[0].predict(df['error_quantile'])

    return df

In [ ]:
def nll(mu,sigma,y):
    eps = 1e-10
    return np.log(eps+sigma**2)/2 + ((y-mu)**2)/(eps+2*sigma**2)

In [ ]:
def nll_floored(y_pred,y_gt):  # only for training of parametric uncertainty model
    mu    = y_pred[:,0]
    sigma = y_pred[:,1]
    y_gt  = torch.squeeze(y_gt)
    
    nll = torch.log(sigma**2)/2 + ((y_gt-mu)**2)/(2*sigma**2)
    nll[nll<-100]=-100 # why floor?
    nll = nll.mean()  # why mean? should be sum i guess
    
    return nll

In [ ]:
# a MSE(y_pred, y) + b MSE(|y_MC - y_pred|, |y - y_pred|) + c MSE(y_MC, y)
# Dropout spread is learned to be equal to the residual of the prediction
def sml_loss(net, data,loss_params):
    inputs, labels = data
    alpha, beta, gamma = loss_params
    mse_loss = torch.nn.MSELoss(reduction='mean') 
    
    outputs_no_mc     = net(inputs,drop_bool=False)
    outputs_no_mc_det = outputs_no_mc.detach()
    outputs_mc        = net(inputs)

    loss0 = mse_loss(outputs_no_mc, labels)        

    a_abs = torch.abs(outputs_mc - outputs_no_mc_det) # | y_MC - y_pred|
    b_abs = torch.abs(labels     - outputs_no_mc_det) # | y_gt - y_pred|
    loss1 = mse_loss(a_abs, b_abs)

    loss2 = mse_loss(outputs_mc, labels)  

    loss = alpha * loss0 + beta * loss1 + gamma * loss2

    return loss 

In [ ]:
# retrieves multiple mc outputs
# loss: mean of outputs should be equal to gt (loss0)
# distance of the mc outputs should resemble the distance of mean output to gt (loss1)
def train_second_moments_loss(net,data,loss_params):
    inputs, labels = data
    alpha, beta, gamma = loss_params
    mse_loss = torch.nn.MSELoss(reduction='mean') 
    
    #outputs_no_mc     = net(inputs,drop_bool=False)
    #outputs_no_mc_det = outputs_no_mc.detach()
    outputs_mc_1    = net(inputs)
    outputs_mc_2    = net(inputs)
    outputs_mc_mean = 0.5*(outputs_mc_1 + outputs_mc_2) 

    loss01 = mse_loss(outputs_mc_1,labels)        
    loss02 = mse_loss(outputs_mc_2,labels)
    loss0  = 0.5*(loss01+loss02)
    
    a_abs = torch.abs(outputs_mc_1 - outputs_mc_2)
    b_abs = torch.abs(labels       - outputs_mc_mean)
    loss1 = mse_loss(a_abs, b_abs)

    loss2 = mse_loss(outputs_mc_1, labels)  

    loss = alpha * loss0 + beta * loss1 + gamma * loss2

    return loss 

In [ ]:
def calc_ece(pred_error_quantiles):
    bins = np.linspace(0.1, 0.9, 9)
    n    = len(pred_error_quantiles)
    
    digitized = np.digitize(pred_error_quantiles, bins)
    ece       = np.abs(((pd.Series(digitized).value_counts()/n)-0.1)).sum()
    
    return ece

In [ ]:
def calc_ece_and_iso_reg(pred_error_quantiles):
    bins = np.linspace(-0.0001,1.0001,21)
    rel_freqs = np.zeros(len(bins)-1)
    n = len(pred_error_quantiles)
        
    digitized = np.digitize(pred_error_quantiles, bins)
    digitized = pd.Series(digitized).value_counts()/n
    
    for i in digitized.index:
        rel_freqs[i-1] = digitized[i]
    
    ece = np.abs(rel_freqs-0.05).sum()
    
    model_quantiles = bins[1:]-0.025
    emp_quantiles   = np.add.accumulate(rel_freqs)
    iso_reg         = IsotonicRegression(out_of_bounds='clip').fit(model_quantiles,emp_quantiles)

 

    return ece, iso_reg

In [ ]:
def calc_global_statistics(df):
    
    rmse = np.sqrt((df['pred_residual']**2).mean())
    r2   = r2_score(df['gt'],df['pred_mean'])
    nll  = df['nll'].mean()
    
    ece, _ = calc_ece_and_iso_reg(df['error_quantile']) 
    
    ws_dist = wasserstein_distance(df['pred_residual_normed'],np.random.randn(100000))
    ks_dist = kstest(df['pred_residual_normed'].values,'norm')[0]
    
    res = {'rmse':rmse,'r2':r2,'nll':nll,'ece':ece, 'ks_dist':ks_dist,'ws_dist':ws_dist}
    if ('error_quantile_calibrated' in df.columns.values):
        res['ece_calib'], _ = calc_ece_and_iso_reg(df['error_quantile_calibrated'])
    return res
    
# example for re-scaling
#np.sqrt((df_train['pred_residual']**2).mean())*np.sqrt(y_scaler.var_[0])

In [ ]:
def net_gradient_norm(datapoint,net):
    test_in  = torch.tensor(datapoint,requires_grad=True,dtype=torch.float32)
    test_out = net(test_in)
    return torch.autograd.grad(test_out, test_in)[0].norm().item()

# better use isotropic Gaussian for const density on eps-sphere
def random_perturb_hull(eps,dim):
    per = np.random.uniform(2*eps,5*eps,dim)
    if norm(per) > eps:
        per = eps*per/norm(per)
    return per

def random_perturb_ball(eps,dim):     
    return norm(np.random.multivariate_normal(np.zeros(dim),0.005*eps*np.diag(np.ones(dim))))

# Import data / Toy dataset generation

In [ ]:
# Computes gaussian * sine; Represents the noise/uncertainty of the main polynomial function
def sine_bump(centre, std, amplitude, frequency):
    def sine_bump_instance(x):
        return amplitude * np.exp( -((x-centre)**2) / (2*std**2) ) * np.sin(frequency*x)

    return sine_bump_instance

# third degree polynomial + uncertainty (sine * gaussian)
def poly_fluct(x, centre=-1, std=1, amplitude=4000, frequency=2):
    return 0.01*((5*x)**2-(1*x)**3+sine_bump(centre,std,amplitude,frequency)(x))

# Like poly_fluct but sine has frequency 1 (why does this represent the "mean"?)
def poly_fluct_mean(x, centre=-1, std=10, amplitude=4000, frequency=1):
    return 0.01 * ((5 * x) ** 2 - (1 * x) ** 3 + sine_bump(centre, std, amplitude, frequency)(x))

# Takes the absolute value of the uncertainty curve
def poly_fluct_sigma(x):
    return np.abs(sine_bump(12, 5, 10, 2)(x))

# samples from a gaussian with the third degree polynomial evaluated at x as mean and the absolute uncertainty curve eval. at x as sigma 
def poly_fluct_sigma_fluct_normal(x,sample_size, centre_1=-1, std_1=1, amplitude_1=4000, frequency_1=2, 
                                  centre_2=12, std_2=5, amplitude_2=1200, frequency_2=0.1, added_std=0):
    return 0.01*(np.random.normal(100*poly_fluct(x, centre_1, std_1, amplitude_1, frequency_1),
                                  np.abs(sine_bump(centre_2,std_2,amplitude_2,frequency_2)(x))+added_std,sample_size))

In [ ]:
# reads plain text file without header, seperation by arbitrary number of whitespace
# converts to float
def plain_table_reader(file):
    res = []
    with open(file) as f:
        for line in f:
            str_feats = [feat.strip() for feat in re.split(r'\s+', line)]
            float_feats = [float(feat) for feat in str_feats if len(feat) > 0]
            if len(float_feats) > 0:
                res.append(float_feats)
    return np.asarray(res)

def load_dataset(id):
    
    if id == 'toy':
        lb, ub, size = -15, 20, 1000 #-20, 30, 1000
        x_range = np.linspace(lb, ub, size)
        X = x_range[:, None]
        y = poly_fluct_mean(x_range)
        return X, y
    
    if id == 'toy_hf':
        lb, ub, size = -15, 20, 1000 #-20, 30, 1000
        x_range = np.linspace(lb, ub, size)
        X = x_range[:, None]
        y = poly_fluct_mean(x_range, frequency=3)
        return X, y
    
    if id == 'toy_uniform':
        sample_size = 10
        lb, ub, steps = -15, 30, 2000
        data_range = np.linspace(lb, ub, steps)
        X = np.repeat(data_range, sample_size)[:, None]
        y = np.concatenate([np.random.randn(sample_size) for i in data_range])#[:, None]
        return X, y
    
    if id == 'toy_modulated':
        sample_size = 10
        lb, ub, steps = -15, 15, 2000
        data_range = np.linspace(lb, ub, steps)
        X = np.repeat(data_range, sample_size)[:, None]
        y = np.concatenate([np.random.normal(0,np.exp(-0.02*i**2),sample_size) for i in data_range])
        return X, y
    
    if id == 'toy_noise':
        sample_size = 10
        lb, ub, steps = -15, 30, 1000
        data_range = np.linspace(lb, ub, steps)
        X = np.repeat(data_range, sample_size)[:, None]
        y = np.concatenate([poly_fluct_sigma_fluct_normal(i, sample_size) for i in data_range])[:, None]
        return X, y
    
    if id == 'toy_noise_strong':
        sample_size = 10
        lb, ub, steps = -15, 30, 2000
        data_range = np.linspace(lb, ub, steps)
        X = np.repeat(data_range, sample_size)[:, None]
        y = np.concatenate([poly_fluct_sigma_fluct_normal(i ,sample_size, centre_1=-5, std_1=2, amplitude_1=4000, frequency_1=2,
                                                          amplitude_2=2000, frequency_2=0.1, added_std=80) for i in data_range])[:, None]
        return X, y

    # Features: 13, Points: 506
    if id == 'boston':
        boston = load_boston()
        return boston['data'], boston['target']
    
    # features: 8, points: 20640
    if id == 'california':
        california = fetch_california_housing()
        return california['data'], california['target']
    
    # features: 7, points: 442
    if id == 'diabetes':
        diabetes = load_diabetes()
        return diabetes['data'], diabetes['target']
    
    #http://archive.ics.uci.edu/ml/datasets/Concrete+Compressive+Strength
    # features: 8, points: 1030
    if id == 'concrete':
        concrete = pd.ExcelFile('./data/Concrete_Data.xls').parse()
        concrete = concrete.to_numpy()
        return concrete[:, :-1], concrete[:, -1]
        
    #https://archive.ics.uci.edu/ml/datasets/Energy+efficiency
    # features: 8, points: 768; 2 gt labels (using latter one)
    if id == 'energy':
        energy_n_feat = 8
        energy_n_gt = 2
        energy = pd.ExcelFile('./data/ENB2012_data.xlsx').parse()
        energy = energy.to_numpy()
        assert(energy.shape[1] == (energy_n_feat + energy_n_gt))
        return energy[:, :-energy_n_gt], energy[:, -1] # note: using cooling load gt only #energy[:, -energy_n_gt:]
    
    #https://archive.ics.uci.edu/ml/datasets/abalone
    # features: 8 (using only 7, first feature is ignored), points: 4176
    if id == 'abalone':
        abalone = pd.read_csv('./data/abalone.data')
        abalone = abalone.to_numpy()[:, 1:].astype(np.float64) # ignoring first feature which is categorical
        return abalone[:, :-1], abalone[:, -1]
    
    #https://archive.ics.uci.edu/ml/datasets/Condition+Based+Maintenance+of+Naval+Propulsion+Plants
    #features: 16, points: 11934, has 2 gt labels, using the latter one
    if id == 'naval':
        naval_n_feat = 16
        naval_n_gt = 2
        naval = plain_table_reader('./data/UCI CBM Dataset/data.txt')
        return naval[:, :-naval_n_gt], naval[:, -1] # note: using turbine gt only #naval[:, -naval_n_gt:]
    
    #https://archive.ics.uci.edu/ml/datasets/Combined+Cycle+Power+Plant
    if id == 'power':
        power = pd.ExcelFile('./data/CCPP/Folds5x2_pp.xlsx').parse()
        power = power.to_numpy()
        return power[:, :-1], power[:, -1]
    
    #https://archive.ics.uci.edu/ml/datasets/Physicochemical+Properties+of+Protein+Tertiary+Structure
    if id == 'protein':
        protein = pd.read_csv('./data/CASP.csv')
        protein = protein.to_numpy()
        return protein[:, 1:], protein[:, 0]
    
    #https://archive.ics.uci.edu/ml/datasets/wine+quality
    # features: 11, points: 1599
    if id == 'wine_red':
        wine_red = pd.read_csv('./data/winequality-red.csv', sep=';')
        wine_red = wine_red.to_numpy()
        return wine_red[:, :-1], wine_red[:, -1]
    
    #http://archive.ics.uci.edu/ml/datasets/yacht+hydrodynamics
    # features: 6, points: 308
    if id == 'yacht':
        yacht = plain_table_reader('./data/yacht_hydrodynamics.data')
        return yacht[:, :-1], yacht[:, -1]
    
    #https://archive.ics.uci.edu/ml/datasets/YearPredictionMSD
    # features: 90, points: 515345
    if id == 'year':
        year = pd.read_csv('./data/YearPredictionMSD.txt', header=None)
        year = year.to_numpy()
        return year[:, 1:], year[:, 0]
    
    # features: 81, points: 21263
    if id == 'superconduct':
        superconduct = pd.read_csv('./data/superconduct/train.csv')
        superconduct = superconduct.to_numpy()
        return superconduct[:, :-1], superconduct[:, -1]
        

In [ ]:
# generate idx lists for data split

In [ ]:
def compute_idx_splits(X, y, fold_idxs=None, split_perc=0.8, splits=None):
    
    if fold_idxs is None:
        fold_idxs = list(range(10))
    else:
        fold_idxs = np.array(fold_idxs)
        if np.any((fold_idxs < 0) | (fold_idxs > 9) ):
            raise Exception("Given fold_idxs have to lie in [0, 9]")
    
    res = dict()
    
    n_data = X.shape[0]
    n_test = n_data // 10
    n_train = n_data - n_test
    assert(n_data == len(y))
    
    idxs_random = np.random.choice(n_data, size=n_data, replace=False)
    
    if 'random_folds' in splits:
        folds = []
        for i in fold_idxs:
            start_test = i*n_test
            end_test = start_test + n_test
            folds.append((np.concatenate((idxs_random[0:start_test], idxs_random[end_test:])),
                              idxs_random[start_test:end_test]))
        res['random_folds'] = folds
    
    if 'single_random_split' in splits:
        n_single_train = int(split_perc*n_data)
        res['single_random_split'] = (idxs_random[:n_single_train], idxs_random[n_single_train:])
    
    if 'single_label_split' in splits:
        y_median = np.median(y)
        idxs_lower_half = np.where(y <= y_median)[0]
        idxs_upper_half = np.where(y > y_median)[0]
        res['single_label_split'] = (idxs_lower_half, idxs_upper_half)
    
    if 'label_folds' in splits:
        quantile_fold_range = 1. / 10.
        fold_label = []
        for i in fold_idxs:
            lower_quantile = i * quantile_fold_range
            upper_quantile = lower_quantile + quantile_fold_range
            lower_quantile = np.quantile(y, lower_quantile)
            upper_quantile = np.quantile(y, upper_quantile)
            fold_label.append((np.concatenate((np.where(y < lower_quantile)[0], np.where(y > upper_quantile)[0])),
                               np.where((lower_quantile <= y) & (y <= upper_quantile))[0]))
        res['label_folds'] = fold_label
    
    pca_scaler = StandardScaler()  # each feature centered around mean with std = 1
    X_scaled = pca_scaler.fit_transform(X)

    pca = PCA(n_components=min(X_scaled.shape[1], 5))
    pca.fit(X_scaled)
    projections = np.matmul(X_scaled, pca.components_[0])
    res['projections'] = projections
    
    if 'single_pca_split' in splits:
        projections_median = np.median(projections)
        idxs_lower_pca0 = np.where(projections <= projections_median)[0]
        idxs_upper_pca0 = np.where(projections > projections_median)[0]
        res['single_pca_split'] = (idxs_lower_pca0, idxs_upper_pca0)
    
    if 'pca_folds' in splits:
        quantile_fold_range = 1. / 10.
        fold_pca0 = []
        for i in fold_idxs:
            lower_quantile = i * quantile_fold_range
            upper_quantile = lower_quantile + quantile_fold_range
            lower_quantile = np.quantile(projections, lower_quantile)
            upper_quantile = np.quantile(projections, upper_quantile)
            fold_pca0.append((np.concatenate((np.where(projections < lower_quantile)[0], np.where(projections > upper_quantile)[0])),
                             np.where((lower_quantile <= projections) & (projections <= upper_quantile))[0]))
        res['pca_folds'] = fold_pca0
    
    return res

In [ ]:
def scale_to_standard(X_train, y_train, X_test, y_test):

    X_scaler = StandardScaler()
    y_scaler = StandardScaler()

    X_train = X_scaler.fit_transform(X_train)
    y_train = y_scaler.fit_transform(y_train.reshape(-1,1))

    X_test = X_scaler.transform(X_test)
    y_test = y_scaler.transform(y_test.reshape(-1,1))
    
    return X_train, y_train, X_test, y_test


# Plot generation

In [ ]:
def plot_densitymap(x, y, ax):
    xmin, xmax = x.min(), x.max()
    ymin, ymax = y.min(), y.max()
    x_range, y_range = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
    positions = np.vstack([x_range.ravel(), y_range.ravel()])
    values = np.vstack([x, y])
    kernel = stats.gaussian_kde(values)
    density = np.reshape(kernel(positions).T, x_range.shape)

    ax.imshow(np.rot90(density), cmap=plt.cm.gist_heat_r, extent=[xmin, xmax, ymin, ymax], aspect='equal')
    ax.plot(x, y, 'k.', markersize=1, alpha=0.1)

def plot_results(method_dict, file=None):
    # set min-/max-values for all subplots in the next cell
    
    plt.clf()

    concatted = pd.DataFrame()

    for key in method_dict:
        for i in [0,1]:
            concatted = pd.concat([concatted,method_dict[key][i][['pred_mean','pred_std','pred_residual','pred_residual_normed']]])

    max_pred_mean     = concatted.quantile(0.98)['pred_mean']
    min_pred_mean     = concatted.quantile(0.02)['pred_mean']
    max_pred_std      = concatted.quantile(0.98)['pred_std']
    min_pred_std      = concatted.quantile(0.02)['pred_std']
    max_pred_residual = concatted.quantile(0.98)['pred_residual']
    min_pred_residual = concatted.quantile(0.02)['pred_residual']
    max_pred_residual_normed = concatted['pred_residual_normed'].quantile(0.98)
    min_pred_residual_normed = concatted['pred_residual_normed'].quantile(0.02)

    # visualize all results

    num_methods  = method_dict.__len__()
    method_names = list(method_dict.keys())
    datasets     = ['train','test']
    colors       = ['b','orange']

    fig,ax = plt.subplots(15,num_methods,figsize=(35,40), squeeze=False)
    ax = np.array(ax)
    
    for j,method in enumerate(method_dict):
        for i,df in enumerate(method_dict[method]):
        
            df.plot.scatter(x='gt',y='pred_mean',ax=ax[0,j],color=colors[i])
            df.plot.scatter(x='gt',y='pred_std',ax=ax[1,j],color=colors[i])
            df.plot.scatter(x='gt',y='total_std',ax=ax[2,j],color=colors[i])
            df.plot.scatter(x='gt',y='pred_residual',ax=ax[3,j],color=colors[i])
            df.plot.scatter(x='pred_residual',y='pred_std',ax=ax[4,j],color=colors[i])
            df.plot.scatter(x='pred_residual',y='total_std',ax=ax[5,j],color=colors[i])
            
            df.plot.scatter(x='pca0_projection',y='pred_mean',ax=ax[10,j],color=colors[i])
            df.plot.scatter(x='pca0_projection',y='pred_std',ax=ax[11,j],color=colors[i])
            df.plot.scatter(x='pca0_projection',y='pred_residual',ax=ax[12,j],color=colors[i])
            ax[13,j].hist(df['pred_residual_normed'],bins=30,density=True,color=colors[i])
            df.plot.scatter(x='gt',y='net_gradient_norm',ax=ax[14,j],color=colors[i])

        try:
            plot_densitymap(method_dict[method][0]['pred_residual'], method_dict[method][0]['pred_std'], ax[6, j])
        except Exception:
            print("Exception caught in plot_densitymap, skipping plot ... ", method_dict[method][0]['pred_residual'],  method_dict[method][0]['pred_std'])
        
        try:
            plot_densitymap(method_dict[method][0]['pred_residual'], method_dict[method][0]['total_std'], ax[7, j])
        except Exception:
            print("Exception caught in plot_densitymap, skipping plot ... ", method_dict[method][0]['pred_residual'],  method_dict[method][0]['total_std'])
        
        try:
            plot_densitymap(method_dict[method][1]['pred_residual'], method_dict[method][1]['pred_std'], ax[8, j])
        except Exception:
            print("Exception caught in plot_densitymap, skipping plot ... ", method_dict[method][1]['pred_residual'],  method_dict[method][1]['pred_std'])

        try:
            plot_densitymap(method_dict[method][1]['pred_residual'], method_dict[method][1]['total_std'], ax[9, j])
        except Exception:
            print("Exception caught in plot_densitymap, skipping plot ... ", method_dict[method][1]['pred_residual'],  method_dict[method][1]['total_std'])
        
        line_sigma_1_data =  pd.DataFrame([[x,np.abs(x)] for x in np.linspace(min_pred_residual-0.2,max_pred_residual+0.2,200)])
        line_sigma_3_data =  pd.DataFrame([[x,np.abs(x/3)] for x in np.linspace(min_pred_residual-0.2,max_pred_residual+0.2,200)])
       
        line_sigma_1_data.plot(kind='line',x=0,y=1, color='k',ax=ax[4,j],alpha=1)
        line_sigma_3_data.plot(kind='line',x=0,y=1,color='r',ax=ax[4,j],alpha=1) 
        line_sigma_1_data.plot(kind='line',x=0,y=1,color='k',ax=ax[5,j],alpha=1)
        line_sigma_3_data.plot(kind='line',x=0,y=1,color='r',ax=ax[5,j],alpha=1)
        line_sigma_1_data.plot(kind='line',x=0,y=1,color='k',ax=ax[6,j],alpha=1)
        line_sigma_3_data.plot(kind='line',x=0,y=1,color='r',ax=ax[6,j],alpha=1) 
        line_sigma_1_data.plot(kind='line',x=0,y=1,color='k',ax=ax[7,j],alpha=1)
        line_sigma_3_data.plot(kind='line',x=0,y=1,color='r',ax=ax[7,j],alpha=1)
        line_sigma_1_data.plot(kind='line',x=0,y=1,color='k',ax=ax[8,j],alpha=1)
        line_sigma_3_data.plot(kind='line',x=0,y=1,color='r',ax=ax[8,j],alpha=1) 
        line_sigma_1_data.plot(kind='line',x=0,y=1,color='k',ax=ax[9,j],alpha=1)
        line_sigma_3_data.plot(kind='line',x=0,y=1,color='r',ax=ax[9,j],alpha=1)
        
        ax[0,j].set_ylim([min_pred_mean,max_pred_mean])
        ax[1,j].set_ylim([min_pred_std,max_pred_std])
        ax[2,j].set_ylim([min_pred_std,max_pred_std])
        ax[3,j].set_ylim([min_pred_residual,max_pred_residual])
        ax[4,j].set_xlim([min_pred_residual-0.2,max_pred_residual+0.2])
        ax[4,j].set_ylim([min_pred_std,max_pred_std])
        ax[4,j].set_xlabel('pred_residual')
        ax[5,j].set_xlim([min_pred_residual-0.2,max_pred_residual+0.2])
        ax[5,j].set_ylim([min_pred_std,max_pred_std])
        ax[5,j].set_xlabel('pred_residual')
        
        ax[6,j].set_xlim([min_pred_residual-0.2,max_pred_residual+0.2])
        ax[6,j].set_ylim([min_pred_std,max_pred_std])
        ax[6,j].set_xlabel('pred_residual')
        ax[7,j].set_xlim([min_pred_residual-0.2,max_pred_residual+0.2])
        ax[7,j].set_ylim([min_pred_std,max_pred_std])
        ax[7,j].set_xlabel('pred_residual')
        
        ax[8,j].set_xlim([min_pred_residual-0.2,max_pred_residual+0.2])
        ax[8,j].set_ylim([min_pred_std,max_pred_std])
        ax[8,j].set_xlabel('pred_residual')
        ax[9,j].set_xlim([min_pred_residual-0.2,max_pred_residual+0.2])
        ax[9,j].set_ylim([min_pred_std,max_pred_std])
        ax[9,j].set_xlabel('pred_residual')
        
        ax[11,j].set_ylim([min_pred_mean,max_pred_mean])
        ax[11,j].set_ylim([min_pred_std,max_pred_std])
        ax[12,j].set_ylim([min_pred_residual,max_pred_residual])
        #ax[7,j].set_ylim(-0.05,1)
        ax[13,j].set_xlim([min_pred_residual_normed-5,max_pred_residual_normed+5])
        ax[13,j].set_xlabel('pred_residual_normed')
        ax[13,j].set_ylabel('pdf')
        #ax[7,j].set_yscale('log')

        for k in range(6):
            ax[k,j].set_title(method_names[j]+' (train/test data)')
        
        for k in range(6, 8):
            ax[k, j].set_title(method_names[j] + ' (train data)')
            ax[k, j].legend()
        
        for k in range(8, 10):
            ax[k, j].set_title(method_names[j] + ' (test data)')
            ax[k, j].legend()
        
        for k in range(10, 15):
            ax[k,j].set_title(method_names[j]+' (train/test data)')

    plt.tight_layout()
    
    if file is not None:
        plt.savefig(file)


# Training and evaluation

In [ ]:
def get_net_from_method(method, n_feat, net_params, train_params):
    
    net_params['n_input'] = n_feat
    
    if method == 'mc':  # dropout in all layers, standard mse
        net_params['n_output']       = 1    
        train_params['drop_bool']    = True
        train_params['drop_bool_ll'] = True
        train_params['loss_func']    = torch.nn.MSELoss(reduction='mean')
        #train_params['num_epochs']   = 2000
        train_params['sml_loss_params'] = None

        net = Net(net_params=net_params,train_params=train_params)
        net.to(train_params['device'])
        return net

    elif method == 'mc_ll':  # dropout in last layer, standard mse
        net_params['n_output']       = 1    
        train_params['drop_bool']    = False
        train_params['drop_bool_ll'] = True
        train_params['loss_func']    = torch.nn.MSELoss(reduction='mean')
        #train_params['num_epochs']   = 2000
        train_params['sml_loss_params'] = None

        net = Net(net_params=net_params,train_params=train_params)
        net.to(train_params['device'])
        return net
    
    elif method == 'mc_mod_sml0':
        net_params['n_output']       = 1    
        train_params['drop_bool']    = True
        train_params['drop_bool_ll'] = True
        #train_params['num_epochs']   = 2000
        train_params['loss_func']    = sml_loss
        train_params['sml_loss_params']= [1,0,0]

        net = Net(net_params=net_params,train_params=train_params)
        net.to(train_params['device'])
        return net
    
    elif method == 'mc_mod_sml10':
        net_params['n_output']       = 1    
        train_params['drop_bool']    = True
        train_params['drop_bool_ll'] = True
        #train_params['num_epochs']   = 2000
        train_params['loss_func']    = sml_loss
        train_params['sml_loss_params']= [1,10,0]

        net = Net(net_params=net_params,train_params=train_params)
        net.to(train_params['device'])
        return net

    elif method == 'mc_mod_sml':  # dropout in all layers, sml loss
        net_params['n_output']       = 1    
        train_params['drop_bool']    = True
        train_params['drop_bool_ll'] = True
        #train_params['num_epochs']   = 2000
        train_params['loss_func']    = sml_loss
        train_params['sml_loss_params']= [1,0.5,0]

        net = Net(net_params=net_params,train_params=train_params)
        net.to(train_params['device'])
        return net
    
    elif method == 'mc_mod_sml75':  # dropout in all layers, sml loss
        net_params['n_output']       = 1    
        train_params['drop_bool']    = True
        train_params['drop_bool_ll'] = True
        #train_params['num_epochs']   = 2000
        train_params['loss_func']    = sml_loss
        train_params['sml_loss_params']= [1,0.75,0]

        net = Net(net_params=net_params,train_params=train_params)
        net.to(train_params['device'])
        return net
    
    elif method == 'mc_mod_sml25':  # dropout in all layers, sml loss
        net_params['n_output']       = 1    
        train_params['drop_bool']    = True
        train_params['drop_bool_ll'] = True
        #train_params['num_epochs']   = 2000
        train_params['loss_func']    = sml_loss
        train_params['sml_loss_params']= [1,0.25,0]

        net = Net(net_params=net_params,train_params=train_params)
        net.to(train_params['device'])
        return net
    
    elif method == 'mc_mod_sml1':  # dropout in all layers, sml loss
        net_params['n_output']       = 1    
        train_params['drop_bool']    = True
        train_params['drop_bool_ll'] = True
        #train_params['num_epochs']   = 2000
        train_params['loss_func']    = sml_loss
        train_params['sml_loss_params']= [1,0.1,0]

        net = Net(net_params=net_params,train_params=train_params)
        net.to(train_params['device'])
        return net
    
    elif method == 'mc_mod_sml9':  # dropout in all layers, sml loss
        net_params['n_output']       = 1    
        train_params['drop_bool']    = True
        train_params['drop_bool_ll'] = True
        #train_params['num_epochs']   = 2000
        train_params['loss_func']    = sml_loss
        train_params['sml_loss_params'] = [1,0.9,0]

        net = Net(net_params=net_params,train_params=train_params)
        net.to(train_params['device'])
        return net

    elif method == 'mc_mod_2moments': # dropout in all layers, 2 moments loss
        net_params['n_output']       = 1    
        train_params['drop_bool']    = True
        train_params['drop_bool_ll'] = True
        #train_params['num_epochs']   = 2000
        train_params['loss_func']    = train_second_moments_loss
        train_params['sml_loss_params'] = [1,0.5,0]

        net = Net(net_params=net_params,train_params=train_params)
        net.to(train_params['device'])
        return net

    elif method == 'pu':    # trains mu, sigma uses nll loss
        net_params['n_output']       = 2
        train_params['drop_bool']    = False
        train_params['drop_bool_ll'] = False
        train_params['loss_func']    = nll_floored
        #train_params['num_epochs']   = 2000
        train_params['sml_loss_params'] = None

        net = Net_PU(net_params=net_params,train_params=train_params)
        net.to(train_params['device'])
        return net
        
    elif method == 'de':
        net_params['n_output']       = 1
        train_params['drop_bool']    = False
        train_params['drop_bool_ll'] = False
        train_params['loss_func']    = torch.nn.MSELoss(reduction='mean')
        #train_params['num_epochs']   = 2000
        train_params['sml_loss_params'] = None
        
        net = []
        for i in range(net_params['de_components']):
            net_ = Net(net_params=net_params,train_params=train_params)
            net_.to(train_params['device'])
            net.append(net_)
            
    elif method == 'pu_de':  
        net_params['n_output']       = 2
        train_params['drop_bool']    = False
        train_params['drop_bool_ll'] = False
        train_params['loss_func']    = nll_floored
        #train_params['num_epochs']   = 2000
        train_params['sml_loss_params'] = None

        net = []
        for i in range(net_params['de_components']):
            net_ = Net_PU(net_params=net_params,train_params=train_params)
            net_.to(train_params['device'])
            net.append(net_)

    elif method == 'sml_de':  
        net_params['n_output']       = 1
        train_params['drop_bool']    = True
        train_params['drop_bool_ll'] = True
        train_params['loss_func']    = sml_loss
        #train_params['num_epochs']   = 2000
        train_params['sml_loss_params'] = [1,0.5,0]

        net = []
        for i in range(net_params['de_components']):
            net_ = Net(net_params=net_params,train_params=train_params)
            net_.to(train_params['device'])
            net.append(net_)
            
    return net

In [ ]:
""" Timestamp of the format: hour:minute:second """                  
def timestamp(dt_obj):
    return "%d_%d_%d_%d_%d_%d" % (dt_obj.year, dt_obj.month, dt_obj.day, dt_obj.hour, dt_obj.minute, dt_obj.second)

In [ ]:
available_datasets = {'boston', 'concrete', 'energy', 'abalone', 'naval', 
                      'power', 'protein', 'wine_red', 'yacht', 'year', 
                      'california', 'diabetes', 'superconduct', 'toy', 'toy_noise',
                     'toy_uniform', 'toy_modulated', 'toy_noise_strong', 'toy_hf'}

toy_datasets = {'toy', 'toy_hf', 'toy_noise', 'toy_uniform', 'toy_modulated', 'toy_noise_strong'}
small_datasets = {'toy', 'toy_hf', 'toy_noise', 'toy_uniform', 'toy_modulated', 'toy_noise_strong',
                  'yacht', 'diabetes', 'boston', 'energy', 'concrete', 'wine_red'}
large_datasets = {'abalone', 'naval', 'power', 'superconduct', 'protein'} #'california',
very_large_datasets = {'year'}

In [ ]:
available_splits = {'random_folds', 'single_random_split', 'single_label_split', 'label_folds', 'single_pca_split', 'pca_folds'}

available_methods = {'de','pu','mc_mod_sml','mc_ll','mc', 'mc_mod_sml9','pu_de','sml_de'}

# Base parameters
net_params = {'n_output':1,
            'layer_width':50,
            'num_layers':2,
            'nonlinearity':nn.ReLU(), #tanh,sigmoid
            'init_corrcoef':0.0,
            'de_components': 5} 

train_params = {'device': 'cpu', #torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
              'drop_bool':True,
              'drop_bool_ll':True,
              'drop_p':0.1,
              'num_epochs': 1000,
              'batch_size': 100,
              'learning_rate': 0.001,
              'loss_func':torch.nn.MSELoss(reduction='mean'),
              'sml_loss_params':[1,0.5,0]}

dt_now = dt.datetime.now()
exp_dir = './INSERT/PATH/TO/EXP/DIR/HERE/NAME_%s' % timestamp(dt_now)
os.makedirs(exp_dir, exist_ok=True)

methods = available_methods # use all methods (de, pu, mc, ...)

start_ = t.time()

for dataset_id in available_datasets: # use all datasets
    
    X, y = load_dataset(dataset_id)
    n_feat = X.shape[1]
    
    net_params_ = dict(net_params)
    train_params_ = dict(train_params)
    
    if dataset_id in very_large_datasets:
        splits = compute_idx_splits(X, y, fold_idxs=[0, 3, 5, 7, 9], splits=['random_folds', 'label_folds', 'pca_folds'])
        train_params_['num_epochs'] = 150
        train_params_['batch_size'] = 500
    
    elif dataset_id in large_datasets:
        splits = compute_idx_splits(X, y, fold_idxs=[0, 3, 5, 7, 9], splits=available_splits)
        train_params_['num_epochs'] = 150 
    else:
        splits = compute_idx_splits(X, y, splits=available_splits) # use 10-folds
    
    projections = splits['projections']
    
    for split_mode in splits:
        
        if split_mode == 'projections':
            continue
        
        folds = splits[split_mode]
        
        if (type(folds) == tuple) and (len(folds) == 2):
            folds = [folds]
        
        for fold_idx, (train_idxs, test_idxs) in enumerate(folds):
            
            identifier = 'dataset=%s_splitmode=%s_foldidx=%d' % (dataset_id, split_mode, fold_idx)
            
            X_train = X[train_idxs]
            X_test = X[test_idxs]
            y_train = y[train_idxs]
            y_test = y[test_idxs]
            
            X_train, y_train, X_test, y_test = scale_to_standard(X_train, y_train, X_test, y_test)
            
            # choose a bunch of uncertainty methods and train the respective models 
            method_dict = {}
            method_dict_json = {}
            net_dict = {}
            for method in methods: 
                method_identifier = '%s_method=%s' % (identifier, method)
                print(method_identifier)

                net = get_net_from_method(method, n_feat, net_params_, train_params_)
                print(net_params_, train_params_)
                train_network(net=net,data=[X_train,y_train], train_params=train_params_, method=method)

                iso_reg = []
                df_train = calc_datapoint_statistics(net=net,data=[X_train,y_train],method=method, iso_reg=iso_reg)
                df_test  = calc_datapoint_statistics(net=net,data=[X_test,y_test],  method=method, iso_reg=iso_reg)

                df_train['pca0_projection'] = projections[train_idxs] 
                df_test['pca0_projection']  = projections[test_idxs] 

                method_dict[method] = [df_train,df_test]
                method_dict_json[method] = [df_train.to_json(), df_test.to_json()]
            
                # store model
                if isinstance(net, list):
                    for i, subnet in enumerate(net):
                        net_dict['%s_sub=%d' % (method_identifier, i)] = copy.deepcopy(subnet.state_dict())
                else:
                    net_dict[method_identifier] = copy.deepcopy(net.state_dict())
            
            exp_dataset_dir = '%s/%s' % (exp_dir, dataset_id)
            os.makedirs(exp_dataset_dir, exist_ok=True)
            
            with gzip.open('%s/method_dict_%s.json.zip' % (exp_dataset_dir, identifier), 'wt', encoding='ascii') as fp:
                json.dump(method_dict_json, fp)
                
            plot_results(method_dict, '%s/%s.png' % (exp_dataset_dir, identifier))
            
            # print global statistics for the different methods (for both train and test)
            global_stats = {}
            for method in method_dict:
                global_stats[method] = []
                for i in [0, 1]:
                    global_stats[method].append(calc_global_statistics(method_dict[method][i]))
            
            with gzip.open('%s/global_stats_%s.json.zip' % (exp_dataset_dir, identifier), 'wt', encoding='ascii') as fp:
                json.dump(global_stats, fp)
                
            torch.save(net_dict, '%s/model_%s.pt' % (exp_dataset_dir, identifier))

print(t.time() - start_)